In [1]:
# !pip install -r requirements.txt
# !pip list

import numpy as np
# import pytorch_nndct
import local_utils
from torchvision import datasets
from torch.utils.data import DataLoader, Sampler
from torchvision.transforms import ToTensor
from local_utils import ResidualBlock
from torch import nn

from distutils.util import strtobool
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

from copy import deepcopy
import os
from re import compile as compile_re
from glob import glob as glob_glob
import sys
import time
import datetime
import argparse

from collections import namedtuple
import tqdm
from typing import Dict

from models import *
from test_fun import test

### 1.Dane ewaluacyjne
Definicja danych i dataloaderów.

In [2]:
train_path = "data/PST900_RGBT_Dataset/train"
test_path = "data/PST900_RGBT_Dataset/test"

#train_loader = torch.utils.data.DataLoader(RGBTFolderP(train_path, train=True), batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(RGBTFolderP(test_path), batch_size=1, shuffle=False)

100%|██████████| 288/288 [00:00<00:00, 356.14it/s]


Zapisanie danych testowych w formacie .npz do późniejszej ewaluacji.

In [3]:
quantization_data = []
quantization_labels = []

for _, img, target in test_loader:
    quantization_data.append(img)
    quantization_labels.append(target)

batch_shape = np.shape(quantization_data[0].numpy())
print(batch_shape)

test_X = torch.cat(quantization_data).numpy()
test_Y = torch.cat(quantization_labels).numpy()

np.savez('eval_PST900.npz', data=test_X, targets=test_Y)

print('Done')

(1, 4, 416, 416)
Done


### 2.Inicjalizacja modelu zmiennoprzecinkowego
Tworzymy model i wczytujemy wagi.

In [4]:
device = torch.device("cpu")
model_config_path = "config/yolov3.cfg"
weights_path = "weights/float_133.weightd"

net = Darknet(model_config_path)
net.load_weights_dict(weights_path)
net.to(device)

net.eval()

print(device)
with open("log_net.txt", "w") as f:
    f.write(str(net))


yolo num: 82
Mask: 6,7,8
Anchors: 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
Anchors: [(116, 90), (156, 198), (373, 326)]
Classes: 4
Height: 416
82 [(116, 90), (156, 198), (373, 326)] 4 416
Init done
yolo num: 94
Mask: 3,4,5
Anchors: 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
Anchors: [(30, 61), (62, 45), (59, 119)]
Classes: 4
Height: 416
94 [(30, 61), (62, 45), (59, 119)] 4 416
Init done
yolo num: 106
Mask: 0,1,2
Anchors: 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
Anchors: [(10, 13), (16, 30), (33, 23)]
Classes: 4
Height: 416
106 [(10, 13), (16, 30), (33, 23)] 4 416
Init done
new init 10


/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370120218/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


cpu


In [5]:
class EvalLoader:
    def __init__(self, 
                 batch_size: int = 1, 
                 npz_path: str = 'eval_PST900.npz') -> None:
        data = np.load(npz_path)
        self.data = data['data'].astype(np.float32)
        self.targets = data['targets']
        self.batch_size = batch_size
    
    def __getitem__(self, i):
        if i >= len(self):
            raise StopIteration

        beg = min(i * self.batch_size, self.data.shape[0])
        end = min(beg + self.batch_size, self.data.shape[0])

        return self.data[beg:end, ...], self.targets[beg:end]
    
    def __len__(self):
        return self.data.shape[0] // self.batch_size

In [6]:
loader = EvalLoader()
for batch_i, (imgs, targets) in enumerate(loader):
    print("Batch:", batch_i)
    print("Img:",imgs.shape)
    print("Target:",targets.shape)
    img = imgs
    target = targets
    break

Batch: 0
Img: (1, 4, 416, 416)
Target: (1, 10, 5)


### 3. Ewaluacja modelu zmiennoprzecinkowego
Upewnienie się, że model został wczytany poprawnie.

In [13]:
kwarg_for_test = {'batch_size': 1,
                  'model_config_path': model_config_path,
                  'data_config_path': "config/pst900.data",
                  'class_path': "data/pst900.names",
                  'n_cpu': 0,
                  'img_size': 416,
                  'use_cuda': False,
                  'weights_path' : weights_path,
                  'iou_thres' : 0.5,
                  'conf_thres' : 0.8,
                  'nms_thres' : 0.5}

test(use_train_model=deepcopy(net), use_loader=test_loader, use_parser=False, suppress_return=False, **kwarg_for_test)

{'batch_size': 1, 'model_config_path': 'config/yolov3.cfg', 'data_config_path': 'config/pst900.data', 'class_path': 'data/pst900.names', 'n_cpu': 0, 'img_size': 416, 'use_cuda': False, 'weights_path': 'weights/float_133.weightd', 'iou_thres': 0.5, 'conf_thres': 0.8, 'nms_thres': 0.5}
Argz(batch_size=1, model_config_path='config/yolov3.cfg', data_config_path='config/pst900.data', weights_path='weights/float_133.weightd', class_path='data/pst900.names', iou_thres=0.5, conf_thres=0.8, nms_thres=0.5, n_cpu=0, img_size=416, use_cuda=False)
Compute mAP...


Detecting objects:   0%|          | 0/258 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'new'

In [13]:
@torch.no_grad()
def evaluation(model: nn.Module,
               data_loader: torch.utils.data.DataLoader,
               conf_thres: int = 0.5,
               nms_thres: int = 0.4
               ):
    
    # for i, (X, y_ref) in tqdm.tqdm(enumerate(data_loader),):
    #     y_pred = model(X)
        
    #     # calculate loss
    #     loss = criterion(y_pred, y_ref)
        
    #     # calculate accuracy
    #     accuracy = metric(y_pred, y_ref)

    #     total_loss += loss * y_pred.shape[0]
    #     total_accuracy += accuracy * y_pred.shape[0]
    #     samples_num += y_pred.shape[0]

    num_classes = 4
    img_size = 416
    iou_thres = 0.5

    # all_detections = []
    # all_annotations = []
    
    yoloL = YOLOLayer([(116, 90), (156, 198), (373, 326)], num_classes, img_size)
    yoloM = YOLOLayer([(30, 61), (62, 45), (59, 119)], num_classes, img_size)
    yoloS = YOLOLayer([(10, 13), (16, 30), (33, 23)], num_classes, img_size)    

    print("Compute mAP...")

    all_detections = []
    all_annotations = []

    for batch_i, (imgs, targets) in enumerate(tqdm.tqdm(data_loader, desc="Detecting objects")):

        # imgs = Variable(imgs.type(Tensor))
        # targets = Variable(targets.type(Tensor), requires_grad=False)

        
        # print(imgs[0][2][234][12:18])
        # print(np.mean(imgs[0][2][100]))

        imgs = torch.from_numpy(imgs)
        targets = torch.from_numpy(targets)


        # img = imgs[0].cpu().numpy()
        # img = img[:3] * 255
        # img = np.transpose(img, (1, 2, 0)).astype(np.uint8)

        # target = targets[0].cpu().numpy()

        # print(type(img), img.shape, img.dtype)
        # for t in target:
        #     cls, x, y, w, h = t

        #     if x != 0 and y != 0 and w != 0 and h != 0:
        #         cx = int(x * 416)
        #         cy = int(y * 416)
        #         w = int(w * 416)
        #         h = int(h * 416)

        #         x = cx - w // 2
        #         y = cy - h // 2

        #         img = np.ascontiguousarray(img)

        #         img = cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #         cv.imshow("img", img)
        #         cv.waitKey(0)

        
#         o1, o2, o3 = model(imgs)
# #         print(type(o2), o1.shape, o2.shape, o3.shape)
# #         o1 = o1.permute(0, 3, 1, 2) #shape 1, 27, 13, 13
# #         o2 = o2.permute(0, 3, 1, 2) #shape 1, 27, 26, 26
# #         o3 = o3.permute(0, 3, 1, 2) #shape 1, 27, 52, 52
#         output = []
#         y3 = yoloL(o3)  #shape 1, 8112, 9
# #         print(type(y1), y1.shape, y1[0, 0, :])
#         y2 = yoloM(o2)  #shape 1, 2028, 9
# #         print(type(y2), y2.shape, y2[0, 0, :])
#         y1 = yoloS(o1) #shape 1, 507, 9
# #         print(type(y3), y3.shape, y3[0, 0, :])
#         output.append(y1)
#         output.append(y2)
#         output.append(y3)
#         outputs = torch.cat(output, 1)

        with torch.no_grad():
            # print(imgs.shape)
            output = model(imgs)
            outputs = []
#             print(output[0].shape)
#             print(output[1].shape)
#             print(output[2].shape)
            y1 = yoloL(output[0])
            y2 = yoloM(output[1])
            y3 = yoloS(output[2])
            outputs.append(y1)
            outputs.append(y2)
            outputs.append(y3)
            outputs = torch.cat(outputs, 1)
#             print([type(x) for x in outputs[0]])
#             print([type(x) for x in outputs[1]])
#             print([type(x) for x in outputs[2]])
#             yolo1 = outputs[0][1][0]
#             x1 = outputs[0][0]
#             # print("yolo1", yolo1.anchors, yolo1.num_classes, yolo1.image_dim, x1.shape)
#             yolo1 = outputs[1][1][0]
#             x1 = outputs[1][0]
#             # print("yolo2", yolo1.anchors, yolo1.num_classes, yolo1.image_dim, x1.shape)
#             yolo1 = outputs[2][1][0]
#             x1 = outputs[2][0]
            # print("yolo3", yolo1.anchors, yolo1.num_classes, yolo1.image_dim, x1.shape)
#             outputs = model.process_yolo_layers(outputs)
            # print(outputs.shape)
            # y1 = outputs[0]
            # print(type(y1), y1.shape, y1[0, :])
            # y1 = outputs[1]
            # print(type(y1), y1.shape, y1[0, :])
            # y1 = outputs[2]
            # print(type(y1), y1.shape, y1[0, :])
            outputs = non_max_suppression(outputs, 4, conf_thres=conf_thres, nms_thres=nms_thres)
            # print([x.shape for x in outputs])

        for output, annotations in zip(outputs, targets):
            all_detections.append([np.array([]) for _ in range(num_classes)])
            if output is not None:
                # Get predicted boxes, confidence scores and labels
                pred_boxes = output[:, :5].cpu().numpy()
                scores = output[:, 4].cpu().numpy()
                pred_labels = output[:, -1].cpu().numpy()
#                 pred_boxes = output[:, :5]
#                 scores = output[:, 4]
#                 pred_labels = output[:, -1]

                # Order by confidence
                sort_i = np.argsort(scores)
                pred_labels = pred_labels[sort_i]
                pred_boxes = pred_boxes[sort_i]
                
                # print("Batch:", batch_i)
                # print("pred_boxes:", pred_boxes)
                # print("scores:", scores)
                # print("pred_labels:", pred_labels)

                for label in range(num_classes):
                    all_detections[-1][label] = pred_boxes[pred_labels == label]

            all_annotations.append([np.array([]) for _ in range(num_classes)])
#             print("Anno", annotations.shape, annotations)
            if any(annotations[:, -1] > 0):

                annotation_labels = annotations[annotations[:, -1] > 0, 0].cpu().numpy()
                _annotation_boxes = annotations[annotations[:, -1] > 0, 1:].cpu()

                # Reformat to x1, y1, x2, y2 and rescale to image dimensions
                annotation_boxes = np.empty_like(_annotation_boxes)
                annotation_boxes[:, 0] = _annotation_boxes[:, 0] - _annotation_boxes[:, 2] / 2
                annotation_boxes[:, 1] = _annotation_boxes[:, 1] - _annotation_boxes[:, 3] / 2
                annotation_boxes[:, 2] = _annotation_boxes[:, 0] + _annotation_boxes[:, 2] / 2
                annotation_boxes[:, 3] = _annotation_boxes[:, 1] + _annotation_boxes[:, 3] / 2
                annotation_boxes *= img_size

                for label in range(num_classes):
                    all_annotations[-1][label] = annotation_boxes[annotation_labels == label, :]

    average_precisions = {}
    for label in range(num_classes):
        true_positives = []
        scores = []
        num_annotations = 0

        for i in tqdm.tqdm(range(len(all_annotations)), desc=f"Computing AP for class '{label}'"):
            detections = all_detections[i][label]
            annotations = all_annotations[i][label]

            num_annotations += annotations.shape[0]
            detected_annotations = []

            for *bbox, score in detections:
                scores.append(score)

                if annotations.shape[0] == 0:
                    true_positives.append(0)
                    continue

                overlaps = bbox_iou_numpy(np.expand_dims(bbox, axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation]

                if max_overlap >= iou_thres and assigned_annotation not in detected_annotations:
                    true_positives.append(1)
                    detected_annotations.append(assigned_annotation)
                else:
                    true_positives.append(0)

        # no annotations -> AP for this class is 0
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        true_positives = np.array(true_positives)
        false_positives = np.ones_like(true_positives) - true_positives
        # sort by score
        indices = np.argsort(-np.array(scores))
        false_positives = false_positives[indices]
        true_positives = true_positives[indices]

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives = np.cumsum(true_positives)

        # compute recall and precision
        recall = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision = compute_ap(recall, precision)
        average_precisions[label] = average_precision

    logger = {}
    print("Average Precisions:")
    for c, ap in average_precisions.items():
        print(f"+ Class '{c}' - AP: {ap}")
        logger[c] = ap

    mAP = np.mean(list(average_precisions.values()))
    logger["mAP"] = mAP
    print(f"mAP: {mAP}")

    return logger, mAP

In [14]:
evaluation(net, loader, 0.8, 0.5)

Init done
Init done
Init done
Compute mAP...


Detecting objects:   5%|▍         | 12/258 [00:04<01:31,  2.70it/s]


KeyboardInterrupt: 

In [6]:
class EvalLoader:
    def __init__(self, 
                 batch_size: int = 1, 
                 npz_path: str = 'eval_PST900.npz') -> None:
        data = np.load(npz_path)
        self.data = data['data'].astype(np.float32)
        self.targets = data['targets']
        self.batch_size = batch_size
    
    def __getitem__(self, i):
        if i >= len(self):
            raise StopIteration

        beg = min(i * self.batch_size, self.data.shape[0])
        end = min(beg + self.batch_size, self.data.shape[0])

        return self.data[beg:end, ...], self.targets[beg:end]
    
    def __len__(self):
        return self.data.shape[0] // self.batch_size

In [11]:
loader = EvalLoader()

x = np.arange(0.1, 1.0, 0.1)
y = np.arange(0.1, 1.0, 0.1)

X, Y = np.meshgrid(x, y)

data = np.zeros((X.shape[0], Y.shape[1]))

for conf, nms in zip(X.ravel(), Y.ravel()):
    i = int(conf * 10 - 1)
    j = int(nms * 10 - 1)
    print("Conf:", conf, "NMS:", nms)
    _, mAP = evaluation(net, loader, conf, nms)
    data[i, j] = mAP

import json
with open('mAP_tests.json', 'w') as f:
    json.dump(data.tolist(), f)

Conf: 0.1 NMS: 0.1
Compute mAP...


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 19196.92it/s]


Average Precisions:
+ Class '0' - AP: 0.29913661295744526
+ Class '1' - AP: 0.5760288930342203
+ Class '2' - AP: 0.34402238764197035
+ Class '3' - AP: 0.003970370370370371
mAP: 0.3057895660010016
Conf: 0.2 NMS: 0.1
Compute mAP...


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 41806.92it/s]


Average Precisions:
+ Class '0' - AP: 0.290911944336737
+ Class '1' - AP: 0.5760288930342203
+ Class '2' - AP: 0.3279955024681657
+ Class '3' - AP: 0.0031272727272727272
mAP: 0.2995159031415989
Conf: 0.30000000000000004 NMS: 0.1
Compute mAP...


...

Average Precisions:
+ Class '0' - AP: 0.1334371509670051
+ Class '1' - AP: 0.16293722569490623
+ Class '2' - AP: 0.14360430364614468
+ Class '3' - AP: 0.0020485981308411214
mAP: 0.11050681960972429
Conf: 0.8 NMS: 0.9
Compute mAP...


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 24954.58it/s]


Average Precisions:
+ Class '0' - AP: 0.14619676253339622
+ Class '1' - AP: 0.17294051416776127
+ Class '2' - AP: 0.1488095238095238
+ Class '3' - AP: 0.0017777777777777779
mAP: 0.11743114457211476
Conf: 0.9 NMS: 0.9
Compute mAP...


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 51586.52it/s]

Average Precisions:
+ Class '0' - AP: 0.1491379879767134
+ Class '1' - AP: 0.18535679111827377
+ Class '2' - AP: 0.1454509177972865
+ Class '3' - AP: 0.0016
mAP: 0.1203864242230684


### 4. Kwantyzacja w trybie PTQ (Post Training Quantization)

Etap 1: Kalibracja Vitis AI Quantizer

Etap 2: Ewaluacja

In [16]:
def quantize(float_model:torch.nn.Module, 
             input_shape:tuple,
             quant_dir:str, 
             quant_mode:str, 
             device:torch.device,
             dataloader,
             conf = 0.1,
             nms = 0.1):
    """
    :param float_model: float model with loaded weights
    :param input_shape: shape of input(CH,W,H)
    :param quant_dir: path to directory with quantized model components
    :param quant_mode: quant_mode in ['calib', 'test'] 
    :param data_loader: data_loader - for 'calib' must be batch_size == 1
    """
    print("Before tm")
    tm = local_utils.TimeMeasurement("Quantization", len(dataloader))
    with tm:
        # available in docker or after packaging 
        # vitis-AI-tools/..../pytorch../pytorch_nndct
        # and installing the package
        print("before import")
        from pytorch_nndct.apis import torch_quantizer, dump_xmodel
        # model to device
        print("Before device")
        model = float_model.to(device)

        # Force to merge BN with CONV for better quantization accuracy
        optimize = 1

        rand_in = torch.randn(input_shape)
        print("get qunatizer start")
        try:
            quantizer = torch_quantizer(
                quant_mode, model, rand_in, output_dir=quant_dir, device=device)
        except Exception as e:
            print("exception:")
            print(e)
            return
        print("get qunatizer end")

        print("get quantized model start")
        quantized_model = quantizer.quant_model
        print("get quantized model end")

        # evaluate
#         output = quantized_model(rand_in)
        print("testing st")
        for conf, nms in zip([0.8, 0.6, 0.5, 0.2, 0.3, 0.2, 0.1], [0.5, 0.3, 0.2, 0.5, 0.3, 0.2, 0.1]):
            print("conf:", conf, "nms:", nms)
            evaluation(quantized_model, dataloader, conf, nms)
        print("testing end")

        # export config
        if quant_mode == 'calib':
            print("export config")
            quantizer.export_quant_config()
            print("export config end")
        # export model
        if quant_mode == 'test':
            print("export xmodel")
            quantizer.export_xmodel(deploy_check=False, output_dir=quant_dir)
            print("export xmodel end")
    print(tm)

In [10]:
quantize(float_model=net, 
         input_shape=batch_shape,
         quant_dir='quant_dir',
         quant_mode='calib',
         device=device,
         dataloader=loader
         )

Before tm
before import
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

[VAIQ_NOTE]: Loading NNDCT kernels...
Before device
get qunatizer start

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing Darknet...

[VAIQ_NOTE]: Start to trace model...

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 249/249 [00:00<00:00, 1245.14it/s, OpInfo: name = Darknet/Sequential[module_list]/ModuleList[105]/Conv2d[conv_105]/9783, type = _convolution]            



[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir/Darknet.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.


/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


get quantized model end
testing st
Init done
Init done
Init done
Compute mAP...


...

torch.Size([1, 27, 13, 13])
torch.Size([1, 27, 26, 26])
torch.Size([1, 27, 52, 52])


Detecting objects: 100%|██████████| 258/258 [35:47<00:00,  8.32s/it]


torch.Size([1, 27, 13, 13])
torch.Size([1, 27, 26, 26])
torch.Size([1, 27, 52, 52])


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 32900.50it/s]

Average Precisions:
+ Class '0' - AP: 0.09786155664138904
+ Class '1' - AP: 0.44125535218903333
+ Class '2' - AP: 0.10047965653077043
+ Class '3' - AP: 0.0006153846153846154
mAP: 0.16005298749414434
testing end
export config

[VAIQ_NOTE]: =>Exporting quant config.(quant_dir/quant_info.json)
export config end
Execution time: 35.0:0.0:51:104, processed 258 frames, throughput: 0.1199383824794463 fps.


In [17]:
quantize(float_model=net, 
         input_shape=batch_shape,
         quant_dir='quant_dir', # directory for quantizer results
         quant_mode='test',
         device=device,
         dataloader=loader
         )

Before tm
before import
Before device
get qunatizer start

[VAIQ_NOTE]: Quantization test process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing Darknet...

[VAIQ_NOTE]: Start to trace model...

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 249/249 [00:00<00:00, 1307.79it/s, OpInfo: name = Darknet/Sequential[module_list]/ModuleList[105]/Conv2d[conv_105]/9783, type = _convolution]            



[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir/Darknet.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.
get quantized model end
testing st
conf: 0.8 nms: 0.5
Init done
Init done
Init done
Compute mAP...


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 94459.71it/s]


Average Precisions:
+ Class '0' - AP: 0.06040067883872402
+ Class '1' - AP: 0.33360889642414104
+ Class '2' - AP: 0.06665883458646617
+ Class '3' - AP: 0.0008
mAP: 0.11536710246233281
conf: 0.6 nms: 0.3
Init done
Init done
Init done
Compute mAP...


...

Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 65787.00it/s]


Average Precisions:
+ Class '0' - AP: 0.08706844170358757
+ Class '1' - AP: 0.4294260835546006
+ Class '2' - AP: 0.10747188366615357
+ Class '3' - AP: 0.0013925925925925926
mAP: 0.1563397503792336
conf: 0.1 nms: 0.1
Init done
Init done
Init done
Compute mAP...


Computing AP for class '3': 100%|██████████| 258/258 [00:00<00:00, 30961.36it/s]

Average Precisions:
+ Class '0' - AP: 0.08770905751647373
+ Class '1' - AP: 0.438294162489611
+ Class '2' - AP: 0.10447825205929256
+ Class '3' - AP: 0.001284848484848485
mAP: 0.15794158013755644
testing end
export xmodel

[VAIQ_NOTE]: =>Converting to xmodel ...



[VAIQ_NOTE]: =>Successfully convert 'Darknet' to xmodel.(quant_dir/Darknet_int.xmodel)
export xmodel end
Execution time: 34.0:0.0:43:698, processed 258 frames, throughput: 0.12381831440381136 fps.


Kompilacja modelu

In [12]:
# compile model
!vai_c_xir --xmodel 'quant_dir/Darknet_int.xmodel' --arch arch.json --net_name Darknet_qu --output_dir build

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B4096_MAX_BG2
[UNILOG][INFO] Graph name: Darknet, with op num: 580
[UNILOG][INFO] Begin to compile...
[UNILOG][INFO] Total device subgraph number 5, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta json is saved to "/workspace/build/meta.json"
[UNILOG][INFO] The compiled xmodel is saved to "/workspace/build/Darknet_qu.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is abe338161e3435855b032ed967e566d3, and has been saved to "/workspace/build/md5sum.txt"
